# Test sample_weight, kernels, C, sklearn estimator

# Setup
Uncomment the next cell if STree is not already installed

In [1]:
#
# Google Colab setup
#
#!pip install git+https://github.com/doctorado-ml/stree

In [2]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.estimator_checks import check_estimator
from sklearn.datasets import make_classification, load_iris, load_wine
from sklearn.model_selection import train_test_split
from stree import Stree
import time

In [3]:
import os
if not os.path.isfile('data/creditcard.csv'):
    !wget --no-check-certificate --content-disposition http://nube.jccm.es/index.php/s/Zs7SYtZQJ3RQ2H2/download
    !tar xzf creditcard.tgz

In [4]:
random_state=1

def load_creditcard(n_examples=0):
    import pandas as pd
    import numpy as np
    import random
    df = pd.read_csv('data/creditcard.csv')
    print("Fraud: {0:.3f}% {1}".format(df.Class[df.Class == 1].count()*100/df.shape[0], df.Class[df.Class == 1].count()))
    print("Valid: {0:.3f}% {1}".format(df.Class[df.Class == 0].count()*100/df.shape[0], df.Class[df.Class == 0].count()))
    y = df.Class
    X = df.drop(['Class', 'Time', 'Amount'], axis=1).values
    if n_examples > 0:
        # Take first n_examples samples
        X = X[:n_examples, :]
        y = y[:n_examples, :]
    else:
        # Take all the positive samples with a number of random negatives
        if n_examples < 0:
            Xt = X[(y == 1).ravel()]
            yt = y[(y == 1).ravel()]
            indices = random.sample(range(X.shape[0]), -1 * n_examples)
            X = np.append(Xt, X[indices], axis=0)
            y = np.append(yt, y[indices], axis=0)
    print("X.shape", X.shape, " y.shape", y.shape)
    print("Fraud: {0:.3f}% {1}".format(len(y[y == 1])*100/X.shape[0], len(y[y == 1])))
    print("Valid: {0:.3f}% {1}".format(len(y[y == 0]) * 100 / X.shape[0], len(y[y == 0])))
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=random_state)
    return Xtrain, Xtest, ytrain, ytest

data = load_creditcard(-5000) # Take all true samples with up to 5000 of the others
# data = load_creditcard(5000)  # Take the first 5000 samples
# data = load_creditcard(-1000) # Take 1000 samples

Xtrain = data[0]
Xtest = data[1]
ytrain = data[2]
ytest = data[3]
_, data = np.unique(ytrain, return_counts=True)
wtrain = (data[1] / np.sum(data),  data[0] / np.sum(data))
_, data = np.unique(ytest, return_counts=True)
wtest = (data[1] / np.sum(data),  data[0] / np.sum(data))
# Set weights inverse to its count class in dataset
weights = np.ones(Xtrain.shape[0],)
weights[ytrain==0] = wtrain[0]
weights[ytrain==1] = wtrain[1]
weights_test = np.ones(Xtest.shape[0],)
weights_test[ytest==0] = wtest[0]
weights_test[ytest==1] = wtest[1]
print(weights[:4], weights_test[:4])

Fraud: 0.173% 492
Valid: 99.827% 284315
X.shape (5492, 28)  y.shape (5492,)
Fraud: 9.141% 502
Valid: 90.859% 4990
[0.09183143 0.09183143 0.09183143 0.09183143] [0.09041262 0.09041262 0.09041262 0.09041262]


# Tests

## Test sample_weights
Compute accuracy with weights in samples. The weights are set based on the inverse of the number of samples of each class

In [5]:
C = 23
print("Accuracy of Train without weights", Stree(C=C, random_state=1).fit(Xtrain, ytrain).score(Xtrain, ytrain))
print("Accuracy of Train with    weights", Stree(C=C, random_state=1).fit(Xtrain, ytrain, sample_weight=weights).score(Xtrain, ytrain))
print("Accuracy of Tests without weights", Stree(C=C, random_state=1).fit(Xtrain, ytrain).score(Xtest, ytest))
print("Accuracy of Tests with    weights", Stree(C=C, random_state=1).fit(Xtrain, ytrain, sample_weight=weights).score(Xtest, ytest))

Accuracy of Train without weights 0.9851716961498439
Accuracy of Train with    weights 0.986732570239334
Accuracy of Tests without weights 0.9866504854368932
Accuracy of Tests with    weights 0.9781553398058253


## Test accuracy with different kernels
Compute accuracy on train and test set with default hyperparmeters of every kernel

In [6]:
random_state=1
for kernel in ['linear', 'rbf', 'poly']:
    now = time.time()
    clf = Stree(C=7, kernel=kernel, random_state=random_state).fit(Xtrain, ytrain)
    accuracy_train = clf.score(Xtrain, ytrain)
    accuracy_test = clf.score(Xtest, ytest)
    time_spent = time.time() - now
    print(f"Time: {time_spent:.2f}s\tKernel: {kernel}\tAccuracy_train: {accuracy_train}\tAccuracy_test: {accuracy_test}")


Time: 26.03s	Kernel: linear	Accuracy_train: 0.9851716961498439	Accuracy_test: 0.9866504854368932
Time: 0.54s	Kernel: rbf	Accuracy_train: 0.9947970863683663	Accuracy_test: 0.9878640776699029
Time: 0.43s	Kernel: poly	Accuracy_train: 0.9960978147762747	Accuracy_test: 0.9854368932038835


## Test diferent values of C

In [7]:
t = time.time()
for C in (.001, .01, 1, 5, 17):
    clf = Stree(C=C, kernel="linear", random_state=random_state)
    clf.fit(Xtrain, ytrain)
    print(f"************** C={C} ****************************")
    print(f"Classifier's accuracy (train): {clf.score(Xtrain, ytrain):.4f}")
    print(f"Classifier's accuracy (test) : {clf.score(Xtest, ytest):.4f}")
    print(clf)
    print(f"**************************************************")
print(f"{time.time() - t:.4f} secs")

************** C=0.001 ****************************
Classifier's accuracy (train): 0.9828
Classifier's accuracy (test) : 0.9848
root feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.4426 counts=(array([0, 1]), array([3491,  353]))
root - Down, <cgaf> - Leaf class=0 belief= 0.981716 impurity=0.1317 counts=(array([0, 1]), array([3490,   65]))
root - Up, <cgaf> - Leaf class=1 belief= 0.996540 impurity=0.0333 counts=(array([0, 1]), array([  1, 288]))

**************************************************
************** C=0.01 ****************************
Classifier's accuracy (train): 0.9834
Classifier's accuracy (test) : 0.9854
root feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.4426 counts=(array([0, 1]), array([3491,  353]))
root - Down, <cgaf> - Leaf class=0 belief= 0.982269 impurity=0.1285 counts=(array([0, 1]), array([3490,   63]))
root - Up

## Test iterator
Check different weays of using the iterator

In [8]:
#check iterator
for i in list(clf):
    print(i)

root feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.4426 counts=(array([0, 1]), array([3491,  353]))
root - Down feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.1205 counts=(array([0, 1]), array([3488,   58]))
root - Down - Down, <cgaf> - Leaf class=0 belief= 0.983921 impurity=0.1188 counts=(array([0, 1]), array([3488,   57]))
root - Down - Up, <pure> - Leaf class=1 belief= 1.000000 impurity=0.0000 counts=(array([1]), array([1]))
root - Up feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.0812 counts=(array([0, 1]), array([  3, 295]))
root - Up - Down, <pure> - Leaf class=0 belief= 1.000000 impurity=0.0000 counts=(array([0]), array([3]))
root - Up - Up, <pure> - Leaf class=1 belief= 1.000000 impurity=0.0000 counts=(array([1]), array([295]))


In [9]:
#check iterator again
for i in clf:
    print(i)

root feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.4426 counts=(array([0, 1]), array([3491,  353]))
root - Down feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.1205 counts=(array([0, 1]), array([3488,   58]))
root - Down - Down, <cgaf> - Leaf class=0 belief= 0.983921 impurity=0.1188 counts=(array([0, 1]), array([3488,   57]))
root - Down - Up, <pure> - Leaf class=1 belief= 1.000000 impurity=0.0000 counts=(array([1]), array([1]))
root - Up feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.0812 counts=(array([0, 1]), array([  3, 295]))
root - Up - Down, <pure> - Leaf class=0 belief= 1.000000 impurity=0.0000 counts=(array([0]), array([3]))
root - Up - Up, <pure> - Leaf class=1 belief= 1.000000 impurity=0.0000 counts=(array([1]), array([295]))


## Test STree is a sklearn estimator

In [10]:
# Make checks one by one
c = 0
checks = check_estimator(Stree(), generate_only=True)
for check in checks:
    c += 1
    print(c, check[1])
    check[1](check[0])

1 functools.partial(<function check_no_attributes_set_in_init at 0x125acaee0>, 'Stree')
2 functools.partial(<function check_estimators_dtypes at 0x125ac7040>, 'Stree')
3 functools.partial(<function check_fit_score_takes_y at 0x125ac2ee0>, 'Stree')
4 functools.partial(<function check_sample_weights_pandas_series at 0x125ac0820>, 'Stree')
5 functools.partial(<function check_sample_weights_not_an_array at 0x125ac0940>, 'Stree')
6 functools.partial(<function check_sample_weights_list at 0x125ac0a60>, 'Stree')
7 functools.partial(<function check_sample_weights_shape at 0x125ac0b80>, 'Stree')
8 functools.partial(<function check_sample_weights_invariance at 0x125ac0ca0>, 'Stree')
9 functools.partial(<function check_estimators_fit_returns_self at 0x125aca040>, 'Stree')
10 functools.partial(<function check_estimators_fit_returns_self at 0x125aca040>, 'Stree', readonly_memmap=True)
11 functools.partial(<function check_complex_data at 0x125ac0e50>, 'Stree')
12 functools.partial(<function check_dt

In [11]:
# Check if the classifier is a sklearn estimator
check_estimator(Stree())

## Compare to SVM

In [12]:
svc = SVC(C=7, kernel='rbf', gamma=.001, random_state=random_state)
clf = Stree(C=17, kernel='rbf', gamma=.001, random_state=random_state)
svc.fit(Xtrain, ytrain)
clf.fit(Xtrain, ytrain)
print("== Not Weighted ===")
print("SVC train score ..:", svc.score(Xtrain, ytrain))
print("STree train score :", clf.score(Xtrain, ytrain))
print("SVC test score ...:", svc.score(Xtest, ytest))
print("STree test score .:", clf.score(Xtest, ytest))
svc.fit(Xtrain, ytrain, weights)
clf.fit(Xtrain, ytrain, weights)
print("==== Weighted =====")
print("SVC train score ..:", svc.score(Xtrain, ytrain))
print("STree train score :", clf.score(Xtrain, ytrain))
print("SVC test score ...:", svc.score(Xtest, ytest))
print("STree test score .:", clf.score(Xtest, ytest))
print("*SVC test score ..:", svc.score(Xtest, ytest, weights_test))
print("*STree test score :", clf.score(Xtest, ytest, weights_test))

== Not Weighted ===
SVC train score ..: 0.9825702393340271
STree train score : 0.9841311134235172
SVC test score ...: 0.9830097087378641
STree test score .: 0.9848300970873787
==== Weighted =====
SVC train score ..: 0.9786680541103018
STree train score : 0.9802289281997919
SVC test score ...: 0.9805825242718447
STree test score .: 0.9817961165048543
*SVC test score ..: 0.9439939825655582
*STree test score : 0.9476832429673473


In [13]:
print(clf)

root feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.4426 counts=(array([0, 1]), array([3491,  353]))
root - Down, <cgaf> - Leaf class=0 belief= 0.990520 impurity=0.0773 counts=(array([0, 1]), array([3448,   33]))
root - Up, <cgaf> - Leaf class=1 belief= 0.881543 impurity=0.5249 counts=(array([0, 1]), array([ 43, 320]))



## Test max_features

In [14]:
for max_features in [None, "auto", "log2", 7, .5, .1, .7]:
    now = time.time()
    print("*"*40)
    clf = Stree(random_state=random_state, max_features=max_features)
    clf.fit(Xtrain, ytrain)
    print(f"max_features {max_features} = {clf.max_features_}")
    print("Train score :", clf.score(Xtrain, ytrain))
    print("Test score .:", clf.score(Xtest, ytest))
    print(f"Took {time.time() - now:.2f} seconds")

****************************************
max_features None = 28
Train score : 0.9846514047866806
Test score .: 0.9866504854368932
Took 10.18 seconds
****************************************
max_features auto = 5
Train score : 0.9836108220603538
Test score .: 0.9842233009708737
Took 5.22 seconds
****************************************
max_features log2 = 4
Train score : 0.9791883454734651
Test score .: 0.9793689320388349
Took 2.05 seconds
****************************************
max_features 7 = 7
Train score : 0.9737252861602498
Test score .: 0.9739077669902912
Took 2.86 seconds
****************************************
max_features 0.5 = 14
Train score : 0.981789802289282
Test score .: 0.9824029126213593
Took 48.35 seconds
****************************************
max_features 0.1 = 2
Train score : 0.9638397502601457
Test score .: 0.9648058252427184
Took 0.35 seconds
****************************************
max_features 0.7 = 19
Train score : 0.9841311134235172
Test score .: 0.98604368